# ML-Workflow

In diesem Projekt untersuchen wir, wie verschiedene Faktoren den Umsatz eines Restaurants beeinflussen und erstellen ein Modell zur Vorhersage des Umsatzes basierend auf mehreren Merkmalen. 

Der Datensatz enthält diverse Informationen über jedes Restaurant, wie die Lage, das Angebot, die Bewertung, die Marketingausgaben und viele weitere.

> **Hinweis**: Der verwendete Datensatz kann [hier](https://www.kaggle.com/datasets/anthonytherrien/restaurant-revenue-prediction-dataset/data) heruntergeladen werden.<br>
> **Hinweis**: Des Weiteren wird dieser [Blog-Post](https://heartbeat.fritz.ai/a-practical-guide-to-feature-engineering-in-python-8326e40747c8) empfohlen. 

## 1. Datenvorbereitung und Bereinigung

#### 1.1 Daten laden

In [1]:
import pandas as pd

In [2]:
datset_filepath = 'data/restaurant_data.csv'
df_restaurants  = pd.read_csv(datset_filepath)

FileNotFoundError: [Errno 2] No such file or directory: 'data/restaurant_data.csv'

In [ ]:
df_restaurants.head(5)

#### 1.2 Datenbereinigung

In [ ]:
nan_values = df_restaurants.isna().sum().sum()
print(f'Der Datensatz enthält {nan_values} NaN-Werte.')

if nan_values > 0:
    df_restaurants = df_restaurants.dropna()
    print('NaN-Werte wurden entfernt.')

#### 1.3 Kurze und knappe Explorative Datenanalyse (EDA)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_restaurants.hist(figsize=(15, 15), bins=50, xlabelsize=12, ylabelsize=12)
plt.tight_layout() # optional to improve presentation of plots (overlap of axis labels)
plt.show()

In [ ]:
df_restaurants.describe()

#### 1.4 Feature-Label-Split

In [ ]:
# Label und wichtige Features definieren
label    = 'Revenue'
features = ['Location', 
            'Cuisine', 
            'Rating', 
            'Average Meal Price', 
            'Marketing Budget', 
            'Social Media Followers', 
            'Chef Experience Years', 
            'Ambience Score', 
            'Service Quality Score', 
            'Parking Availability'
]

# Features und Label extrahieren
X = df_restaurants[features]
y = df_restaurants[label]

## 2. Feature Engineering

#### 2.1 One-Hot-Encoding

Die Features `Location`, `Cuisine` und `Parking Availability` sind kategorisch und können daher nicht direkt im Modell verwendet werden. Mithilfe von One-Hot-Encoding (OHE) können wir diese Features jedoch in numerische Variablen umwandeln und die Daten für das Modell transformieren.

In [ ]:
%%script false --no-raise-error # Damit diese Zelle nicht ausgeführt wird

# Beispiel an 'Cuisine'
X_copy = X.copy()
X_copy.drop('Cuisine', axis=1, inplace=True)

# One-Hot-Encoding
one_hot_feature = pd.get_dummies(X['Cuisine'], prefix='Cuisine')

# Zusammenführen
X_copy_cuisine_encoded = pd.concat([X_copy, one_hot_feature], axis=1)
X_copy_cuisine_encoded.head(5)

In [ ]:
# ODER ganz direkt in einem Schritt:
X_encoded = pd.get_dummies(X, columns=['Location', 'Cuisine', 'Parking Availability'])
X_encoded.head(5)

#### 2.2 Weitere Features aus bestehenden Features herleiten

In [ ]:
X_encoded['Total Reservations'] = df_restaurants['Weekend Reservations'] + df_restaurants['Weekday Reservations']

## 3. Daten-Skalierung und Train-Test-Split

#### 3.1 Skalierung der Daten

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler().fit(X_encoded)

X_scaled = scaler.transform(X_encoded)
X_scaled

In [ ]:
%%script false --no-raise-error

# ODER:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X_encoded)
X_scaled


#### 3.2 Train-Test-Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
print(f'X_train: {X_train.shape}, y_train: {y_train.shape}\nX_test:  {X_test.shape}, y_test:  {y_test.shape}')

## 4. Modelltraining und Validierung

#### 4.1 Modell trainieren und Vorhersagen treffen

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

#### 4.3 Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, predictions)
print(f'Der MSE auf den Testdaten beträgt: {mse}')

In [ ]:
import math

rmse = math.sqrt(mse)
print(f'Wir liegen im Schnitt um {rmse:.2f} neben dem tatsächlichen Wert.')

## 5. Analyse und Visualisierung

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.scatter(y_test, predictions)

min_val = min(y_test.min(), predictions.min())
max_val = max(y_test.max(), predictions.max())
plt.plot([min_val, max_val], [min_val, max_val], color='r', linestyle='--', label='Wahre Werte')

plt.xlabel('Tatsächlicher Umsatz')
plt.ylabel('Vorhergesagter Umsatz')
plt.title('Tatsächlicher Umsatz vs. Vorhergesagter Umsatz')
plt.show()

In [ ]:
residuals = y_test - predictions
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=30, edgecolor='k')
plt.xlabel('Residuen')
plt.ylabel('Häufigkeit')
plt.title('Histogramm der Residuen')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(predictions, residuals)
plt.xlabel('Vorhergesagte Werte')
plt.ylabel('Residuen')
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Residuen vs. Vorhergesagte Werte')
plt.show()

# Verbesserungen?

#### Polynomiale Features (siehe *ML_VL_04_LineareRegression.pdf*, Folie 46 ff.)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Die Features polynomiell erweitern
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_encoded)

# Die polynomiell erweiterten Features skalieren
sclaer_poly = StandardScaler()
X_poly_scaled = sclaer_poly.fit_transform(X_poly)

# Train-Test-Split
X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(X_poly_scaled, y, test_size=0.2, random_state=42)

# Das Modell trainieren
model_poly = LinearRegression()
model_poly.fit(X_train_poly, y_train_poly)

# Vorhersagen
predictions_poly = model_poly.predict(X_test_poly)

# RMSE berechnen
mse_poly = mean_squared_error(y_test_poly, predictions_poly)
rmse_poly = math.sqrt(mse_poly)

print(f'Der RMSE hat sich von {rmse:.2f} auf {rmse_poly:.2f} verbessert.')

## Visualiserung der Performance aller Modelle im Vergleich und der wahren Werte

In [ ]:
plt.figure(figsize=(10, 6))

plt.scatter(y_test, predictions, label='Lineare Regression', color='blue')
plt.scatter(y_test_poly, predictions_poly, label='Polynomiale Regression', color='green')

# Wahre Werte
min_val = min(y_test.min(), predictions.min())
max_val = max(y_test.max(), predictions.max())
plt.plot([min_val, max_val], [min_val, max_val], color='r', linestyle='--', label='Wahre Werte')

plt.xlabel('Tatsächlicher Umsatz')
plt.ylabel('Vorhergesagter Umsatz')
plt.title('Tatsächlicher Umsatz vs. Vorhergesagter Umsatz')
plt.legend()
plt.show()